In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency

In [3]:
data = pd.read_csv('churn_analysis.csv')

### 1.  Проверить для пользовотелей контрольной группы гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. 

Формируем контрольную выборку

In [4]:
data_control = data[data['treatment'] == 1]
data_control.shape

(1097, 23)

Для построения сопряженных таблиц выберем уникальные значения признаков state и churn

In [5]:
state_ = data_control['state']
churn_ = data_control['churn']

In [6]:
state_uniq = state_.unique()
len(state_uniq)

51

In [7]:
churn_uniq = churn_.unique()

Формируем таблицы сопряженности

In [8]:
contingency_tables = list()

In [9]:
state_len = len(state_uniq)

In [10]:
for i in range(state_len - 1):
    for j in range(i + 1, state_len):
        contingency_data = {}
        for churn in churn_uniq:
            contingency_data.update({churn: [len(data_control[(state_ == state_uniq[i]) & (churn_ == churn)]),
                                             len(data_control[(state_ == state_uniq[j]) & (churn_ == churn)])]})
        contingency_table = pd.DataFrame(contingency_data, index=[state_uniq[i], state_uniq[j]])
        contingency_tables.append(contingency_table)

In [11]:
contingency_tables[0]

,False.,True.
KS,18,7
OH,22,2


In [12]:
len(contingency_tables)

1275

Используем Критерий Фишера 

In [13]:
alpha = 0.05
deviation = 0
for i in contingency_tables:
    oddsratio, p_value = fisher_exact(i)
    if p_value < alpha:
        deviation += 1

In [14]:
deviation

10

Применение Критерия Фишера показало, что количество достигаемых уровней значимости, меньших alpha, равно 10.

### 2. Корреляции Пирсона и Спирмена между day_calls и mes_estim

In [15]:
pearson_corr = data['day_calls'].corr(data['mes_estim'], method='pearson')
pearson_corr

-0.051794350587572612

In [16]:
stats.pearsonr(data.day_calls, data.mes_estim)

(-0.051794350587572625, 0.0027798836869756707)

In [17]:
spearman_corr = data['day_calls'].corr(data['mes_estim'], method='spearman')
spearman_corr

0.043349880533927444

In [18]:
stats.spearmanr(data.day_calls, data.mes_estim)

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

    На основе полученных значений можно сделать вывод:
        Критерий Пирсона:
           Гипотеза будет выполняться, если значения у alpha будут маленькие. Данный вывод можно сделать на основании значе- ния 0.00278, которое получено с помощью метода stats.pearsonr и является pvalue
        Критерий Спирмена:
           Гипотеза не будет отвергаться, при выполнении неравества alpha < pvalue, то есть при значениях alpha меньше 0.0123

### 3. Посчитайте значение коэффицента корреляции Крамера между штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Проверьте гипотезу об отсутствии связи между этими признаками.

Для того,что бы посчитать значение коэффицента корреляции Крамера между штатом (state) и оттоком пользователей (churn) составим таблицу сопряженности

In [19]:
con_table = pd.crosstab(data_control.state, data_control.churn)

In [20]:
con_table.head()

churn,False.,True.
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5


In [21]:
chi2, p, dof, expected = chi2_contingency(con_table)
chi2

44.052712366925483

In [22]:
cramer = np.sqrt(chi2 / len(data_control))
cramer

0.20039321502033319

В первом пунтке мы проверяли гипотезу с помощью Критерия Фишера. На 10 случаях гипотеза при точности 0.05 не выполняется. Гипотеза заключалась в том, что отказ не зависит от штата, получается, что в 10 зависит. Так как мы получили большой коэффициент корреляции, то гипотезу не отвергаем. 

### 4. Проведите анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1). Что можно сказать об этих двух методах (treatment = 0, treatment = 2)?

In [23]:
tr_0 = data[data['treatment'] == 0]
tr_1 = data[data['treatment'] == 1]
tr_2 = data[data['treatment'] == 2]

#### 4.1 treatment = 0 в сравнении с treatment = 1

In [24]:
data0_1 = pd.crosstab(pd.concat([tr_0, tr_1]).treatment, pd.concat([tr_0, tr_1]).churn)

In [25]:
data0_1

churn,False.,True.
treatment,,
0,968,165
1,917,180


In [26]:
p = chi2_contingency(data0_1,  correction = False)
print ('chi2 = {}\np_value = {} '.format(p[0],p[1]))

chi2 = 1.4512270505058507
p_value = 0.22833116390451058 


p_value = 0.228: только при очень больших альфа отвергаем гипотезу о несвязанности признаков. Данный метод неэффективный. 

#### 4.2 treatment = 2 в сравнении с treatment = 1

In [27]:
data1_2 = pd.crosstab(pd.concat([tr_1, tr_2]).treatment, pd.concat([tr_1, tr_2]).churn)
data1_2

churn,False.,True.
treatment,,
1,917,180
2,965,138


In [28]:
p = chi2_contingency(data1_2, correction = False)
print ('chi2 = {}\np_value = {} '.format(p[0],p[1]))

chi2 = 6.755085962437478
p_value = 0.009348084294451088 


p_value = 0.009: данное значение мало, поэтому гипотезу о несвязности отвергаем даже при малых альфа. Данный метод эффективен. 